In [210]:
import pandas as pd
import requests

from register_new_users import MemberManager

In [211]:
access_data_path = "access_tokens.tsv"

access_data = pd.read_csv(access_data_path, sep="\t")
access_data.set_index("ID", inplace=True)
access_data.head(3)

manager = MemberManager()

In [212]:
responses = manager.get_responses()
responses["First_name"] = responses["First Name"]
responses["Last_name"] = responses["Last Name"]
responses = responses.drop(["First Name", "Last Name"], axis=1)
responses.head(3)

,URL,First_name,Last_name
ACYDBNhd0eMPbHlqgTFJCDtm3mJqItmRS5t7fBe8xJ4DZMOyDic8F6q4Vr2pKd92ekKbwFs,http://localhost/exchange_token?state=&code=7f...,Alfie,Hoskings
ACYDBNhG4cmj4aWP1gjh4EnI8klWwhz724kuV2YquXwcXgXtQ84_8rh_dbZfQKy_6NKIQBA,http://localhost/exchange_token?state=&code=9b...,Dobbie,The elf
ACYDBNiaMpqjmIxPsxa6uCLPhnk5q9GjltX3kHGzrF1TjTekRBInTHFQTG7e4kczT7OT6JY,http://localhost/exchange_token?state=&code=b0...,Ben,Hoskings


In [213]:
def get_access_tokens(code):
    response = requests.post(
        url='https://www.strava.com/oauth/token',
        data={
            'client_id': 111130,
            'client_secret': 'b7fca9288af24aaa2f86f1b82f70f2baa54daeb2',
            'code': code,
            'grant_type': 'authorization_code'
        }
    )

    tokens = response.json()

    try:
        athlete_tokens = {
            "ID": tokens["athlete"]["id"],
            "Access_token": tokens["access_token"],
            "Refresh_token": tokens["refresh_token"]
        }

    except:
        athlete_tokens = tokens

    return athlete_tokens

In [214]:
# tokens = {"ID": 38803221, "Access_token": 'cee165ac6b8f4c4db712e81accbfafeed0cc5f7a', "Refresh_token": '840befc02a4dca632ce7439fc082942c282f4e70'}

for response_id in responses.index:
    response = responses.loc[response_id]
    # print(response.to_dict())
    try:
        parts = response["URL"].split("&")
        code = parts[1][5:]
        tokens = get_access_tokens(code)
        if tokens:
            if "message" in tokens.keys():
                print("link extraction failed")
            else:
                responses.loc[response_id, "ID"] = tokens["ID"]
                responses.loc[response_id, "Refresh_token"] = tokens["Refresh_token"]
                responses.loc[response_id, "Access_token"] = tokens["Access_token"]
                responses.loc[response_id, "Access_token"] = tokens["Access_token"]
                responses = responses.loc[:, ~responses.columns.str.contains('^Unnamed')]
                
    except Exception as e:
        # raise e
        print("link extraction failed")
        
if (responses.columns.str.contains('ID')).any():
    responses = responses.set_index("ID")
else:
    responses = None

if responses is not None:    
    responses.head()

link extraction failed
link extraction failed


In [215]:
access_data = pd.concat([access_data, responses], join='inner')
access_data = access_data.loc[access_data.index.dropna()]
# access_data.join(responses, on="ID", how="left")
access_data.head(10)

,First_name,Last_name,Access_token,Refresh_token
ID,,,,
963223.0,Neil,Hoskings,50a242b9cee0831afd21c4c69b50efcbd77cb2a7,0aa21162002e540df4dae47840ddc42f17745c75
98588012.0,Becky,Hoskings,779a0e50d83c0a058242421898b973fb36dd3b58,23d2c1b7a49bc26cd22afb819b3a19d6db7ffcf3
38781770.0,Clark,Roberts,f7292115d74a60e843bc89e71df889cf25d1157f,ab54a8bd816db93297ef7cf7022ffa76e6efc36b
33370063.0,Tom,Beesley,e082250e89b5fd14702a192e5472cec508463e62,5a14aa37c6b245c36dccdb5b5c4467f1996d327c
71665908.0,Sophie,Hurst,303457d3e3a5eb30b5ee8babc59fd40dcfbe09d5,bbe9aa060ed7423b16777886520dfc554bd6052e
14188705.0,Wais,Yip,a697cc6c8273752f78aa8f930de8f059de671afd,995a9f58911040bf48924824e8e9bd8e843b5097
NaN,Alfie,Hoskings,NaN,NaN
38807221.0,Dobbie,The elf,6259ba1ba174925249ff64e263e8602e58bbdcdc,840befc02a4dca632ce7439fc082942c282f4e70
NaN,Ben,Hoskings,NaN,NaN


In [218]:

access_data.to_csv("access_data_update.tsv", sep="\t", index=True)

In [3]:
import requests
import json
url = "https://eu-west-2.aws.data.mongodb-api.com/app/data-izribar/endpoint/data/v1/action/findOne"

payload = json.dumps({
    "collection": "athlete_data",
    "database": "godiva_data",
    "dataSource": "Cluster0"
})

headers = {
  'Content-Type': 'application/json',
  'Access-Control-Request-Headers': '*',
  'api-key': 'b8bRma7UNr1eGbaQjBb2aftslsjCe3SGltB4acl8o5jC0hrNstH2O1zmaV8QT8zx',
}

response = requests.request("POST", url, headers=headers, data=payload)

print(response.text)


{"document":{"_id":"66585b3e1941040e53220a27","first_name":"Ben","last_name":"Hoskings","dob":"27/07/2002","strava_id":38807221,"group":"REPS"}}
